실습 #1: gradio의 chatbot interface를 사용하여 ChatGPT와 스트리밍하면서 대화하는 인터페이스를 구성하시오.
실습 #2: 만든 어플리케이션을 멀티턴으로 수정하시오.


In [10]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def gpt_reponse(text):
    stream = openai.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": "너는 친절한 친구같은 AI야."
        },
        {
          "role": "user",
          "content": text
        }
      ],
      temperature=0.8,
      max_tokens=1024,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stream=True
    )
    return stream


In [11]:
# 스트림

import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        g_res = gpt_reponse(history[-1][0])
        history[-1][1] = ""
        for char in g_res:
            if char.choices[0].delta.content is not None:
                history[-1][1] += char.choices[0].delta.content
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    
demo.queue()
demo.launch(server_name='0.0.0.0')

/tmp/ipykernel_99678/3546856197.py:8: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://0.0.0.0:7898
* To create a public link, set `share=True` in `launch()`.
